<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/difference_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("s-nlp/paradetox")

In [3]:
import difflib

def get_diff_ops(original, edited):
    try:
        ops = []
        s = difflib.SequenceMatcher(None, original.split(), edited.split())
        for tag, i1, i2, j1, j2 in s.get_opcodes():
            if tag == 'delete':
                ops.append(("delete", " ".join(original.split()[i1:i2]), i1))
            elif tag == 'insert':
                ops.append(("insert", " ".join(edited.split()[j1:j2]), j1))
            elif tag == 'replace':
                ops.append(("replace", " ".join(original.split()[i1:i2]), i1, " ".join(edited.split()[j1:j2])))
        return ops
    except Exception as e:
        print(f"Error with original: {original} and edited: {edited}. Error: {e}")
        return []

In [4]:
def apply_diff_ops(example):
    original = example['en_toxic_comment']
    edited = example['en_neutral_comment']

    ops_list = get_diff_ops(original, edited)

    formatted_ops = []
    for op in ops_list:
        if op[0] == 'replace':
            formatted_ops.append({
                'operation': op[0],
                'content': op[1],
                'position': op[2],
                'replacement_content': op[3]
            })
        else:
            formatted_ops.append({
                'operation': op[0],
                'content': op[1],
                'position': op[2],
                'replacement_content': None
            })

    example['edit_ops'] = formatted_ops
    return example

In [5]:
dataset_with_ops = dataset.map(apply_diff_ops)

Map:   0%|          | 0/19744 [00:00<?, ? examples/s]

In [6]:
for i in range(5):
    print(dataset_with_ops["train"][i])

{'en_toxic_comment': 'he had steel balls too !', 'en_neutral_comment': 'he was brave too!', 'edit_ops': [{'content': 'had steel balls too !', 'operation': 'replace', 'position': 1, 'replacement_content': 'was brave too!'}]}
{'en_toxic_comment': 'dude should have been taken to api , he would be right at home with all the other knuckleheads there', 'en_neutral_comment': 'It would have been good if he went to api. He would fit in.', 'edit_ops': [{'content': 'dude should', 'operation': 'replace', 'position': 0, 'replacement_content': 'It would'}, {'content': 'taken', 'operation': 'replace', 'position': 4, 'replacement_content': 'good if he went'}, {'content': 'api , he', 'operation': 'replace', 'position': 6, 'replacement_content': 'api. He'}, {'content': 'be right at home with all the other knuckleheads there', 'operation': 'replace', 'position': 10, 'replacement_content': 'fit in.'}]}
{'en_toxic_comment': 'im not gonna sell the fucking picture , i just want to contribute to the fucking a

In [8]:
dataset_with_ops.push_to_hub("difference_analysis")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]